In [ ]:
import math
import pandas as pd
from openalgo import api
import os

OPEN_ALGO_API_KEY = os.getenv('OPEN_ALGO_API_KEY')

# Config
API_KEY = OPEN_ALGO_API_KEY
HOST = 'http://127.0.0.1:5000'
SYMBOL = 'BANKNIFTY MAY FUT'
EXCHANGE = 'NSE'
START_DATE = '2025-04-01'
END_DATE = '2025-04-30'
INTERVALS = ['1m']#, '5m', '10m', '15m']
OUTPUT_FILE = 'volume_changes.xlsx'
PERCENTAGE_THRESHOLD = 0.03

# Initialize client
client = api(api_key=API_KEY, host=HOST)

def fetch_data(symbol, exchange, interval, start_date, end_date):
    return client.history(
        symbol=symbol,
        exchange=exchange,
        interval=interval,
        start_date=start_date,
        end_date=end_date
    )

def get_relative_volume_changes(data):
    if isinstance(data, dict) and "candles" in data:
        candles = data["candles"]
    elif isinstance(data, pd.DataFrame):
        dates = data.index.tolist()
        volumes = data["volume"].tolist()
        closes = data["close"].tolist()
        opens = data["open"].tolist()
    elif isinstance(data, list):
        candles = data
    else:
        raise ValueError("Unsupported format for data")

    def sign(x):
        return (x > 0) - (x < 0)
    

    result = {}

    if 'candles' in locals():
        for i in range(1, len(candles)):
            try:
                date1 = str(candles[i - 1][0])[:19]
                date2 = str(candles[i][0])[:19]
                vol1 = candles[i - 1][5]
                vol2 = candles[i][5]
                diff = vol2 - vol1
                pct = round(diff / vol1, 4) if vol1 else 0
                d1= candles[i - 1][4]-candles[i - 1][1]
                d2= candles[i][4]-candles[i][1]
                rg=math.sign(d1)==math.sign(d2)
                
                result[f"{date2} - {date1}"] = {
                    'vol1': vol1,
                    'vol2': vol2,
                    'diff': diff,
                    'percentage': pct,
                    'd1': d1,
                    'd2': d2,
                    'rg': rg
                }
            except Exception:
                continue
    else:
        for i in range(1, len(volumes)):
            try:
                date1 = str(dates[i - 1])[:19]
                date2 = str(dates[i])[:19]
                vol1 = volumes[i - 1]
                vol2 = volumes[i]
                diff = vol2 - vol1
                pct = round(diff / vol1, 4) if vol1 else 0
                os = f"{opens[i - 1]}, {opens[i]}"
                cs = f"{closes[i - 1]}, {closes[i]}"
                d1= closes[i-1]-opens[i - 1]
                d2= closes[i]-opens[i]
                rg=sign(d1)*sign(d2)
                
                result[f"{date2} - {date1}"] = {
                    'vol1': vol1,
                    'vol2': vol2,
                    'diff': diff,
                    'percentage': pct,
                    'opens': os,
                    'closes': cs,
                    'd1': d1,
                    'd2': d2,
                    'rg': rg
                }
            except Exception(e):
                # Handle the exception as needed, e.g., log it or print a message
                print(f"Error processing data for interval {interval} at index {i}: {e}")
                continue

    return result

def filter_changes(changes, threshold=0.03):
    # return {k: v for k, v in changes.items() if abs(v['percentage']) <= threshold}
    return {k: v for k, v in changes.items() if abs(v['percentage']) <= threshold and v['rg'] > 0}
from datetime import datetime

def add_peak_info(filtered_changes, full_data):
    if isinstance(full_data, dict) and "candles" in full_data:
        candles = full_data["candles"]
    elif isinstance(full_data, pd.DataFrame):
        candles = list(zip(
            full_data.index,
            full_data['open'],
            full_data['high'],
            full_data['low'],
            full_data['close'],
            full_data['volume']
        ))
    else:
        raise ValueError("Unsupported format for full_data")

    # Build timestamp → index lookup for fast access
    time_to_index = {str(c[0])[:19]: i for i, c in enumerate(candles)}
    trigger_keys = list(filtered_changes.keys())
    trigger_indices = [time_to_index.get(k.split(' - ')[0]) for k in trigger_keys]

    for idx, key in enumerate(trigger_keys):
        start_idx = trigger_indices[idx]
        end_idx = trigger_indices[idx + 1] if idx + 1 < len(trigger_indices) else len(candles)

        if start_idx is None:
            continue

        peak_high = float('-inf')
        peak_time = None

        for i in range(start_idx, end_idx):
            high = candles[i][2]
            if high > peak_high:
                peak_high = high
                peak_time = str(candles[i][0])[:19]

        trigger_time = key.split(' - ')[0]
        try:
            duration = (
                datetime.fromisoformat(peak_time) - datetime.fromisoformat(trigger_time)
            ).total_seconds() / 60  # in minutes
        except:
            duration = None

        filtered_changes[key]['peak_high'] = peak_high
        filtered_changes[key]['peak_time'] = peak_time
        filtered_changes[key]['duration_to_peak_min'] = duration

    return filtered_changes

# Write to Excel with multiple sheets
with pd.ExcelWriter(OUTPUT_FILE) as writer:
    for interval in INTERVALS:
        raw_data = fetch_data(SYMBOL, EXCHANGE, interval, START_DATE, END_DATE)
        changes = get_relative_volume_changes(raw_data)
        filtered = filter_changes(changes, PERCENTAGE_THRESHOLD)
        enriched = add_peak_info(filtered, raw_data)

        df = pd.DataFrame.from_dict(enriched, orient='index').reset_index()
        df.rename(columns={'index': 'date_range'}, inplace=True)
        df.to_excel(writer, sheet_name=interval, index=False)


IndexError: At least one sheet must be visible

In [3]:

# Convert to DataFrame
table_df = pd.DataFrame.from_dict(changes, orient='index')


# Filter changes with abs(percentage) >= 0.3
filtered_changes = {
    k: v for k, v in changes.items() if abs(v['percentage']) <= 0.03
}

# filter rg with only positive values
filtered_changes = {
    k: v for k, v in filtered_changes.items() if v['rg'] > 0
}

# Convert to DataFrame
table_df = pd.DataFrame.from_dict(filtered_changes, orient='index')

# Optional: Reset index to make the date range a column
table_df.reset_index(inplace=True)
table_df.rename(columns={'index': 'date_range'}, inplace=True)

table_df

,date_range,vol1,vol2,diff,percentage,opens,closes,d1,d2,rg,peak_high,peak_time,duration_to_peak_min
0,2025-04-01 09:28:00 - 2025-04-01 09:27:00,42911,42551,-360,-0.0084,"1271.3, 1271.95","1271.95, 1273.5",0.65,1.55,1,1277.90,2025-04-01 09:49:00,21.0
1,2025-04-01 10:29:00 - 2025-04-01 10:28:00,55878,57060,1182,0.0212,"1262.9, 1261.45","1261.75, 1261.1",-1.15,-0.35,1,1263.85,2025-04-01 11:33:00,64.0
2,2025-04-01 11:51:00 - 2025-04-01 11:50:00,7506,7588,82,0.0109,"1261.2, 1261.4","1261.4, 1261.6",0.20,0.20,1,1262.85,2025-04-01 12:19:00,28.0
3,2025-04-01 13:34:00 - 2025-04-01 13:33:00,10577,10884,307,0.0290,"1253.85, 1254.4","1254.4, 1255.45",0.55,1.05,1,1255.80,2025-04-01 13:56:00,22.0
4,2025-04-01 15:01:00 - 2025-04-01 15:00:00,59985,61137,1152,0.0192,"1255.1, 1254.8","1254.7, 1254.05",-0.40,-0.75,1,1254.80,2025-04-01 15:01:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2025-04-30 09:40:00 - 2025-04-30 09:39:00,80039,81995,1956,0.0244,"1393.8, 1395.3","1395.2, 1396.3",1.40,1.00,1,1405.50,2025-04-30 09:51:00,11.0
120,2025-04-30 11:45:00 - 2025-04-30 11:44:00,18388,18846,458,0.0249,"1403.0, 1402.9","1402.9, 1402.6",-0.10,-0.30,1,1411.50,2025-04-30 12:21:00,36.0
121,2025-04-30 12:23:00 - 2025-04-30 12:22:00,26864,27016,152,0.0057,"1411.2, 1411.4","1411.3, 1411.5",0.10,0.10,1,1412.40,2025-04-30 12:25:00,2.0
122,2025-04-30 13:48:00 - 2025-04-30 13:47:00,27973,27870,-103,-0.0037,"1410.0, 1409.4","1409.4, 1409.2",-0.60,-0.20,1,1411.20,2025-04-30 14:05:00,17.0
